# FLOPS 分析

- FLOPs（每秒浮点运算次数）通过计算执行的浮点运算次数来衡量神经网络模型的计算复杂度
- 高 FLOPs 表示更密集的计算和能耗

In [ ]:
# pip install -r requirements-extra.txt

In [ ]:
from importlib.metadata import version

pkgs = [
    "thop",
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

thop version: 0.1.1-2209072238
torch version: 2.4.1+cu121


&nbsp;
# 固定批大小的简单基准测试

- 仅前向传播

In [ ]:
import torch
from thop import profile

# 安装说明请参见：
# https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
from llms_from_scratch.ch04 import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # 词汇表大小
    "context_length": 1024,  # 上下文长度
    "drop_rate": 0.0,        # Dropout率
    "qkv_bias": True         # 查询-键-值偏置
}

model_configs = {
    "gpt-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 2
input_tensor = torch.randint(0, 50257, (batch_size, 1024)).to(device)

for size in model_configs:
    BASE_CONFIG.update(model_configs[size])

    model = GPTModel(BASE_CONFIG).bfloat16()
    model.to(device)

    # MACS = 乘法累加运算
    # MACS 通常计算为两个 FLOPS（一个乘法和一个累加）
    macs, params = profile(model, inputs=(input_tensor,), verbose=False)
    flops = 2*macs
    print(f"{size:18}: {flops:.1e} FLOPS")

    del model
    torch.cuda.empty_cache()

&nbsp;
# 自动批大小查找的简单基准测试

- 仅前向传播

In [ ]:
for size in model_configs:
    print(f"\n处理 {size}")
    config = BASE_CONFIG.copy()
    config.update(model_configs[size])

    min_batch_size = 1
    max_batch_size = None
    max_possible_batch_size = 4096

    while min_batch_size <= max_possible_batch_size:
        batch_size = (min_batch_size + max_possible_batch_size) // 2
        try:
            input_tensor = torch.randint(
                0, config["vocab_size"],
                (batch_size, config["context_length"]),
                device=device
            )

            model = GPTModel(config).bfloat16().to(device)

            # MACS = 乘法累加运算
            # MACS 通常计算为两个 FLOPS（一个乘法和一个累加）
            macs, params = profile(model, inputs=(input_tensor,), verbose=False)
            flops = 2 * macs
            print(f"  批大小 {batch_size}: {flops:.1e} FLOPS")

            # 如果成功，尝试更大的批大小
            min_batch_size = batch_size + 1
            max_batch_size = batch_size

            # 清理内存
            del model, input_tensor
            torch.cuda.empty_cache()

        except RuntimeError as e:
            if "out of memory" in str(e):
                # 尝试更小的批大小
                max_possible_batch_size = batch_size - 1

                # 清理内存
                try:
                    del model, input_tensor
                    torch.cuda.empty_cache()
                except NameError:
                    pass
            else:
                raise e

&nbsp;
# 具有自动批大小查找和模型 FLOP 利用率 (MFU) 的基准测试

- 来自 [PaLM 论文](https://arxiv.org/abs/2204.02311) 的模型 FLOPs 利用率 (MFU) 解释

> 我们提出了一个新的效率指标，它与实现无关，允许更清晰地比较系统效率，称为模型 FLOPs 利用率 (MFU)。这是观察到的吞吐量（每秒令牌数）相对于在峰值 FLOPs 下运行的系统理论最大吞吐量的比率。关键是，"理论最大"吞吐量只考虑计算前向+反向传播所需的运算，而不考虑重新计算。


$$\text{MFU} = \frac{\text{观察到的每秒令牌数}}{\text{理论最大每秒令牌数}}$$

其中

$$\text{理论最大每秒令牌数} = \frac{\text{每秒最大 FLOPs}}{\text{每个令牌的总 FLOPs}}$$

以及

$$\text{每秒令牌数} = \frac{\text{批大小} \times \text{序列长度}}{\text{总时间}}$$

- 前向和反向传播

In [ ]:
# GPU 制造商提供的理论最大每秒 FLOPS

flops_per_second = {
    # https://www.techpowerup.com/gpu-specs/h100-pcie-80-gb.c3899
    "H100": {
        torch.float32: 51.22e12,  # NVIDIA H100 FP32 为 51.22 TFLOPs
        torch.float16: 204.9e12,  # NVIDIA H100 FP16 为 204.9 TFLOPs
        torch.bfloat16: 204.9e12
    },
    # https://www.techpowerup.com/gpu-specs/l4.c4091
    "L4": {
        torch.float32: 30.29e12,  # NVIDIA L4 FP32 为 30.29 TFLOPs
        torch.float16: 30.29e12,  # NVIDIA L4 FP16 为 30.29 TFLOPs
        torch.bfloat16: 30.29e12
    },
    # https://www.techpowerup.com/gpu-specs/tesla-t4.c3316
    "T4": {
        torch.float32: 8.1e12,  # NVIDIA T4 FP32 为 8.1 TFLOPs
        torch.float16: 65.13e12,  # NVIDIA T4 FP16 为 65.13 TFLOPs
        torch.bfloat16: 65.13e12
    },
    # https://www.techpowerup.com/gpu-specs/a10g.c3798
    "A10G": {
        torch.float32: 31.52e12,  # NVIDIA A10G FP32 为 31.52 TFLOPs
        torch.float16: 31.52e12,  # NVIDIA A10G FP16 为 31.52 TFLOPs
        torch.bfloat16: 31.52e12
    },
    # https://www.techpowerup.com/gpu-specs/a100-pcie-40-gb.c3623
    "A100": {
        torch.float32: 19.49e12,  # NVIDIA A100 FP32 为 19.49 TFLOPs
        torch.float16: 77.97e12,  # NVIDIA A100 FP16 为 77.97 TFLOPs
        torch.bfloat16: 77.97e12
    },
    # https://www.techpowerup.com/gpu-specs/geforce-rtx-3080.c3621
    "RTX_3080": {
        torch.float32: 29.77e12,  # NVIDIA RTX 3080 FP32 为 29.77 TFLOPs
        torch.float16: 29.77e12,  # NVIDIA RTX 3080 FP16 为 29.77 TFLOPs
        torch.bfloat16: 29.77e12
    },
    # https://www.techpowerup.com/gpu-specs/geforce-rtx-3090.c3622
    "RTX_3090": {
        torch.float32: 35.58e12,  # NVIDIA RTX 3090 FP32 为 35.58 TFLOPs
        torch.float16: 35.58e12,  # NVIDIA RTX 3090 FP16 为 35.58 TFLOPs
        torch.bfloat16: 35.58e12
    }
}

In [ ]:
import time

def get_gpu_model(flops_per_second_dict):
    device_name = torch.cuda.get_device_name(0)
    for model in flops_per_second_dict.keys():
        if model in device_name:
            return model
    return "Unknown"  # 如果没有找到匹配的模型，返回默认值


gpu_model = get_gpu_model(flops_per_second)
print("GPU 模型:", gpu_model)

if gpu_model != "Unknown":

    for size in model_configs:
        print(f"\n处理 {size}")
        config = BASE_CONFIG.copy()
        config.update(model_configs[size])

        min_batch_size = 1
        max_batch_size = None
        max_possible_batch_size = 4096

        while min_batch_size <= max_possible_batch_size:
            batch_size = (min_batch_size + max_possible_batch_size) // 2
            try:
                input_tensor = torch.randint(
                    0, config["vocab_size"],
                    (batch_size, config["context_length"]),
                    device=device
                )

                model = GPTModel(config).bfloat16().to(device)
                model.train()

                # 开始计时
                torch.cuda.synchronize()
                start_time = time.time()

                # 前向 & 反向传播
                output = model(input_tensor)
                loss = output.sum()  # 计算一个虚拟损失
                loss.backward()

                # 结束计时
                torch.cuda.synchronize()
                end_time = time.time()

                total_time_seconds = end_time - start_time

                # 计算前向传播的 FLOPs
                macs, params = profile(model, inputs=(input_tensor,), verbose=False)
                flops_forward = 2 * macs  # 假设一个 MAC 等于两个 FLOPs

                # 估计反向传播的 FLOPs（通常是前向 FLOPs 的 2 倍）
                flops_backward = 2 * flops_forward

                # 前向 + 反向传播的总 FLOPs
                total_flops = flops_forward + flops_backward  # 或者 total_flops = flops_forward * 3

                data_type = next(model.parameters()).dtype
                max_flops_per_second = flops_per_second[gpu_model].get(data_type, 0)

                # 计算每秒令牌数
                tokens_processed = batch_size * config["context_length"]
                tokens_per_second = tokens_processed / total_time_seconds

                # 计算每个令牌的 FLOPs
                flops_per_token = total_flops / tokens_processed

                # 计算理论最大每秒令牌数
                if flops_per_token > 0:
                    theoretical_max_tokens_per_second = max_flops_per_second / flops_per_token
                else:
                    theoretical_max_tokens_per_second = 0  # 避免除零

                # 计算 MFU
                if theoretical_max_tokens_per_second > 0:
                    mfu = tokens_per_second / theoretical_max_tokens_per_second
                else:
                    mfu = 0  # 避免除零

                print(f"  批大小 {batch_size}: 令牌/秒: {tokens_per_second:.2f}, MFU: {mfu:.4f}")

                # 如果成功，尝试更大的批大小
                min_batch_size = batch_size + 1
                max_batch_size = batch_size

                # 清理内存
                del model, input_tensor, output, loss
                torch.cuda.empty_cache()

            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    # 尝试更小的批大小
                    max_possible_batch_size = batch_size - 1

                    # 清理内存
                    try:
                        del model, input_tensor
                        torch.cuda.empty_cache()
                    except NameError:
                        pass
                else:
                    raise e

else:
    print("未知的 GPU 模型。请使用您的 GPU 信息更新 flops_per_second 字典。")

- 值为 1.0 是最好的（等于 100%）
- 注意批大小比之前更小，因为我们这里也执行反向传播，这需要更多内存